In [ ]:
import tensorflow_datasets as tfds  # TFDS to download MNIST.
import tensorflow as tf  # TensorFlow / `tf.data` operations.
from flax import nnx  # The Flax NNX API.
from functools import partial
import jax.numpy as jnp  # JAX NumPy
import jax
import optax
from linearRNN import forward_h
from linearRNN import forward
from linearRNN import init_lru_parameters
from linearRNN import binary_operator_diag
from linearRNN import LRU
import numpy as np
from flax import linen as nn


In [ ]:
rnn=1 #rnn=0:transformation of the inputs with fixed RNN weights, rnn=1: adding the RNN module on the model to learn the weight matrices
mlp=1
hidden_neuron=256
encoded_size=512
hidden_size=256
learning_rate = 0.01
momentum = 0.9
train_steps=1200
eval_every = 100
batch_size=64
r_min = 0
r_max = 1
max_phase = 6.28
method_name="LRU+MLP"
dataset_name="MNIST"
leave_data=1

In [ ]:
#Import data

if dataset_name=="MNIST":
    dataset=tf.keras.datasets.mnist.load_data()
    train=dataset[0]
    test=dataset[1]

    train_x_seq=train[0].shape[0]
    train_x_len=int(jnp.prod(jnp.array(train[0].shape[1:])))
    train_x_size=1
    test_x_seq=test[0].shape[0]
    test_x_len=int(jnp.prod(jnp.array(test[0].shape[1:])))
    test_x_size=1

    train_x=train[0].reshape((train_x_seq,train_x_len,train_x_size))/255
    train_y=train[1].reshape(train_x_seq)
    train_y_class=len(jnp.unique(train_y))

    test_x=test[0].reshape((test_x_seq,test_x_len,test_x_size))/255
    test_y=test[1].reshape(test_x_seq)

if dataset_name=="CIFAR10":
    dataset=tf.keras.datasets.cifar10.load_data()
    train=dataset[0]
    test=dataset[1]

    train_x_seq=train[0].shape[0]
    train_x_len=int(jnp.prod(jnp.array(train[0].shape[1:-1])))
    train_x_size=int(jnp.prod(jnp.array(train[0].shape[-1])))

    test_x_seq=test[0].shape[0]
    test_x_len=int(jnp.prod(jnp.array(test[0].shape[1:-1])))
    test_x_size=int(jnp.prod(jnp.array(train[0].shape[-1])))

    train_x=train[0].reshape((train_x_seq,train_x_len,train_x_size))/255

    train_y=train[1].reshape(train_x_seq)
    train_y_class=len(jnp.unique(train_y))

    test_x=test[0].reshape((test_x_seq,test_x_len,test_x_size))/255
    test_y=test[1].reshape(test_x_seq)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
if rnn:
    train_hidden_state=train_x
    test_hidden_state=test_x
else:
    param=init_lru_parameters(hidden_size,train_x_len,r_min=0.999,r_max=0.9999,max_phase=6.28)
    param2=init_lru_parameters(hidden_size,train_x_size,r_min=0.999,r_max=0.9999,max_phase=6.28)
    train_hidden_state=jnp.real(forward_h(param,train_x))
    test_hidden_state=jnp.real(forward_h(param2,test_x))

In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((jnp.real(train_hidden_state),jnp.array(train_y,dtype=int)))
test_ds=tf.data.Dataset.from_tensor_slices((jnp.real(test_hidden_state),jnp.array(test_y,dtype=int)))

train_ds = train_ds.repeat().shuffle(100)
# Group into batches of `batch_size` and skip incomplete batches, prefetch the next sample to improve latency.
train_ds = train_ds.batch(batch_size, drop_remainder=True).take(train_steps).prefetch(1)
# Group into batches of `batch_size` and skip incomplete batches, prefetch the next sample to improve latency.
test_ds = test_ds.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
print(train_hidden_state.shape)
print(test_hidden_state.shape)

In [ ]:
from flax import nnx
import optax
from flax.nnx.nn.recurrent import LSTMCell,GRUCell
import copy
import random
class MLP(nnx.Module):
  #DON'T FORGET TO CHANGE THE MODEL NAME BEFORE RUNNING
  #According to the scheme of the paper (Figure 1), input_size=M, encoded_size=H,layer_dim=number of neurons in MLP, out_dim=number of classes
  def __init__(self, token_size, token_len,encoded_dim,hidden_dim, layer_dim, out_dim, rngs: nnx.Rngs):

    #linear encoder
    self.lin_encoder = nnx.Linear(in_features=token_size, out_features=encoded_dim,rngs=rngs)

    #LRU+MLP block
    self.rnn = LRU(in_features=encoded_dim, hidden_features=hidden_dim, r_min=r_min,r_max=r_max,max_phase=max_phase)
    self.linear1 = nnx.Linear(in_features=encoded_dim, out_features=layer_dim, rngs=rngs)
    self.linear2 = nnx.Linear(in_features=layer_dim,out_features=encoded_dim,rngs=rngs)

    #Linear layers
    self.weight = nnx.Param(jnp.array(np.random.rand(token_len,1)))
    self.bias = nnx.Param(jnp.array(np.random.rand(encoded_dim,1)))
    self.weight2 = nnx.Param(jnp.array(np.random.rand(out_dim,encoded_dim)))
    self.bias2 = nnx.Param(jnp.array(np.random.rand(out_dim,1)))
    self.out_dim = out_dim

    
  @nnx.vmap(in_axes=(None,0)) 
  def __call__(self, x):
    x = self.lin_encoder(x)
    y = x.copy()
    #LRU+MLP block
    if rnn:
      x = self.rnn(x)
    x = self.linear1(x)
    x = nnx.softplus(x)
    x = self.linear2(x)

    x += y #Skip connection
    x = x.T@self.weight + self.bias #project from L*H to H*1
    x = self.weight2@x + self.bias2#project from H*1 to out_dim
    return x.reshape(self.out_dim)


model = MLP(train_x_size,train_x_len,encoded_size,hidden_size, hidden_neuron, train_y_class, rngs=nnx.Rngs(0))  # eager initialization

nnx.display(model)

In [ ]:
for step,batch in enumerate(train_ds.as_numpy_iterator()):
    batch1=batch
    break

a=model(batch1[0])

In [ ]:
import optax

optimizer = nnx.Optimizer(model, optax.adamw(learning_rate, momentum))
metrics = nnx.MultiMetric(
  accuracy=nnx.metrics.Accuracy(),
  loss=nnx.metrics.Average('loss'),
)

nnx.display(optimizer)

In [ ]:
import jax
def loss_fn(model: MLP, batch):
  logits = model(batch[0])
  loss = optax.softmax_cross_entropy_with_integer_labels(
    logits=logits, labels=batch[1]
  ).mean()
  #print(logits.shape)
  #print(batch[1].shape)
  return loss, logits

@nnx.jit
def train_step(model: MLP, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
  """Train for a single step."""
  grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch[1])  # In-place updates.
  optimizer.update(grads)  # In-place updates.
  predicted_labels = jnp.argmax(logits, axis=-1)
  actual_labels = batch[1]
  #jax.debug.print("Predictions: {}",predicted_labels[:5].astype(int))
  #jax.debug.print("Actual Labels: {}",actual_labels[:5].astype(int))
  
@nnx.jit
def eval_step(model: MLP, metrics: nnx.MultiMetric, batch):
  loss, logits = loss_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch[1])  # In-place updates.

In [ ]:
#Train the model + evaluation with the test data
metrics_history = {
    'train_loss': [],
    'train_accuracy': [],
    'test_loss': [],
    'test_accuracy': [],
}

for step, batch in enumerate(train_ds.as_numpy_iterator()):
  # Run the optimization for one step and make a stateful update to the following:
  # - The train state's model parameters
  # - The optimizer state
  # - The training loss and accuracy batch metrics
  train_step(model, optimizer, metrics, batch)

  if step > 0 and (step % eval_every == 0 or step == train_steps - 1):  # One training epoch has passed.
    # Log the training metrics.
    for metric, value in metrics.compute().items():  # Compute the metrics.
      metrics_history[f'train_{metric}'].append(value)  # Record the metrics.
    metrics.reset()  # Reset the metrics for the test set.

    # Compute the metrics on the test set after each training epoch.
    for test_batch in test_ds.as_numpy_iterator():
      eval_step(model, metrics, test_batch)

    # Log the test metrics.
    for metric, value in metrics.compute().items():
      metrics_history[f'test_{metric}'].append(value)
    metrics.reset()  # Reset the metrics for the next training epoch.

    print(
      f"[train] step: {step}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
      f"accuracy: {metrics_history['train_accuracy'][-1] * 100}"
    )
    print(
      f"[test] step: {step}, "
      f"loss: {metrics_history['test_loss'][-1]}, "
      f"accuracy: {metrics_history['test_accuracy'][-1] * 100}"
    )

In [ ]:
#Save the training results into csv
import pandas as pd

if leave_data:
    data=pd.DataFrame({"step":np.arange(eval_every,train_steps+eval_every,eval_every),"train_loss":metrics_history['train_loss'],
                       "test_loss":metrics_history['test_loss'],"train_accuracy":metrics_history['train_accuracy'],
                       "test_accuracy":metrics_history['test_accuracy']})
    if mlp:
        data.to_csv(method_name+"_enc"+str(encoded_size)+"_nr"+str(hidden_neuron)+"_d"+str(hidden_size)+"_"+dataset_name+"_step"+str(train_steps)+"r_min_"+str(r_min)+"r_max"+str(r_max)+".csv")
    #else:
    #    data.to_csv(method_name+"_d"+str(hidden_size)+"_"+dataset_name+"_step"+str(train_steps)+".csv")

In [ ]:
#Plot the loss
import matplotlib.pyplot as plt

plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['train_loss'],label="train loss")
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['test_loss'],label="test loss")
if mlp:
    plt.title("Train loss of "+dataset_name+" dataset with "+method_name+
              ", \nhidden dimension="+str(hidden_size)+", number of neuron="+str(hidden_neuron))
#else:
#    plt.title("Train loss of MNIST dataset with GRU+MLP, \nhidden dimension="+str(hidden_size))
plt.xlabel("Training step")
plt.ylabel("Train loss (cross entropy)")
plt.legend()
if leave_data:
    plt.savefig("loss_"+method_name+"_"+str(encoded_size)+"_"+str(hidden_neuron)+"_"+dataset_name+"_step"+str(train_steps)+"r_min_"+str(r_min)+"r_max"+str(r_max)+".jpg")
plt.show()

In [ ]:
#Plot the accuracy
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['train_accuracy'],label="train")
plt.plot(np.arange(eval_every,train_steps+eval_every,eval_every),metrics_history['test_accuracy'],label="test")
plt.title("Accuracy of "+dataset_name+" dataset with "+method_name+"MLP, \nhidden dimension="+
          str(hidden_size)+", number of neuron="+str(hidden_neuron))
plt.ylabel("Accuracy")
plt.legend()
if leave_data:
    plt.savefig("accuracy_"+method_name+"_"+str(encoded_size)+"_"+str(hidden_neuron)+"_"+dataset_name+"_step"+str(train_steps)+"r_min_"+str(r_min)+"r_max"+str(r_max)+".jpg")
plt.show()